In [1]:
import os
SPARK_HOME="/opt/spark"
os.environ["PYSPARK_PYTHON"] = "python3"
# os.environ["PYSPARK_PYTHON"]="python"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"]="notebook"
os.environ["SPARK_HOME"] = SPARK_HOME
os.environ["SPARK_CLASSPATH"] = "/opt/spark/jars/sqljdbc4.jar"

 

from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("Clustering") \
        .getOrCreate()

In [2]:
#from pyspark.sql import SQLContext
# sqlContext = SQLContext(spark)
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
data_customer=spark.read.csv('/home/rupeshr/Desktop/TSA_Python/clustering_dataset/archive/CC GENERAL.csv', header=True, inferSchema=True)
data_customer.printSchema()

root
 |-- CUST_ID: string (nullable = true)
 |-- BALANCE: double (nullable = true)
 |-- BALANCE_FREQUENCY: double (nullable = true)
 |-- PURCHASES: double (nullable = true)
 |-- ONEOFF_PURCHASES: double (nullable = true)
 |-- INSTALLMENTS_PURCHASES: double (nullable = true)
 |-- CASH_ADVANCE: double (nullable = true)
 |-- PURCHASES_FREQUENCY: double (nullable = true)
 |-- ONEOFF_PURCHASES_FREQUENCY: double (nullable = true)
 |-- PURCHASES_INSTALLMENTS_FREQUENCY: double (nullable = true)
 |-- CASH_ADVANCE_FREQUENCY: double (nullable = true)
 |-- CASH_ADVANCE_TRX: integer (nullable = true)
 |-- PURCHASES_TRX: integer (nullable = true)
 |-- CREDIT_LIMIT: double (nullable = true)
 |-- PAYMENTS: double (nullable = true)
 |-- MINIMUM_PAYMENTS: double (nullable = true)
 |-- PRC_FULL_PAYMENT: double (nullable = true)
 |-- TENURE: integer (nullable = true)



In [4]:
data_customer=data_customer.na.drop()

In [5]:
from pyspark.ml.feature import VectorAssembler
data_customer.columns
assemble=VectorAssembler(inputCols=[
 'BALANCE',
 'BALANCE_FREQUENCY',
 'PURCHASES',
 'ONEOFF_PURCHASES',
 'INSTALLMENTS_PURCHASES',
 'CASH_ADVANCE',
 'PURCHASES_FREQUENCY',
 'ONEOFF_PURCHASES_FREQUENCY',
 'PURCHASES_INSTALLMENTS_FREQUENCY',
 'CASH_ADVANCE_FREQUENCY',
 'CASH_ADVANCE_TRX',
 'PURCHASES_TRX',
 'CREDIT_LIMIT',
 'PAYMENTS',
 'MINIMUM_PAYMENTS',
 'PRC_FULL_PAYMENT',
 'TENURE'], outputCol='features')
assembled_data=assemble.transform(data_customer)
assembled_data.show(2)


+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+--------------------+
|CUST_ID|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|            features|
+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+--------------------+
| C10001|  40.900749|       

In [6]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)
data_scale_output.show(2)

+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+--------------------+--------------------+
|CUST_ID|    BALANCE|BALANCE_FREQUENCY|PURCHASES|ONEOFF_PURCHASES|INSTALLMENTS_PURCHASES|CASH_ADVANCE|PURCHASES_FREQUENCY|ONEOFF_PURCHASES_FREQUENCY|PURCHASES_INSTALLMENTS_FREQUENCY|CASH_ADVANCE_FREQUENCY|CASH_ADVANCE_TRX|PURCHASES_TRX|CREDIT_LIMIT|   PAYMENTS|MINIMUM_PAYMENTS|PRC_FULL_PAYMENT|TENURE|            features|        standardized|
+-------+-----------+-----------------+---------+----------------+----------------------+------------+-------------------+--------------------------+--------------------------------+----------------------+----------------+-------------+------------+-----------+----------------+----------------+------+--------

In [7]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    output=KMeans_fit.transform(data_scale_output)

KeyboardInterrupt: 

In [10]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):
    
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    
    output=KMeans_fit.transform(data_scale_output)
    
    WCSS=KMeans_algo.computeCost(data_scale_output)
    print(WCSS)
    
    score=evaluator.evaluate(output)
    
    silhouette_score.append(score)
    
    print("Silhouette Score:",score)

AttributeError: 'KMeans' object has no attribute 'computeCost'

In [9]:
centers = KMeans_algo.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

AttributeError: 'KMeans' object has no attribute 'clusterCenters'

In [10]:
#Visualizing the silhouette scores in a plot
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,10),silhouette_score)
ax.set_xlabel(‘k’)
ax.set_ylabel(‘cost’)

SyntaxError: invalid character in identifier (<ipython-input-10-8581a9cd84cc>, line 5)

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel

# Load and parse the data
data = sc.textFile("data/mllib/kmeans_data.txt")
parsedData = data.map(lambda line: array([float(x) for x in line.split(' ')]))

# Build the model (cluster the data)
clusters = KMeans.train(parsedData, 2, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [ ]:
class clusteroptnum(APIView):
    def post(self,request):
        try:
            connections.close_all()
            t1 = datetime.now()
            data = request.data
            module = data.get('module', 'da')
            filename = data['filename']
            username = data['username']
            stat = Statistics.objects.get(file_name=filename)
            colmap = ast.literal_eval(stat.descriptive_stats)
            for i in colmap:
                if i['ignored'] == 'no':
                    if i['missing_values'] != 0:
                        return Response({"status": "missingvalues"})
            spark_df = read_file(username,filename,module)
            column = spark_df.columns
            column = get_unignored(filename, column)  # actual columns in terms of c0,c1 after unignore
            spark_df = spark_df.select(column)  # here the dataset will contain the columns which are unignored
            catcols, numcols = find_numerical_categorical_columns(spark_df)
            for catcol in catcols:
                categories = []
                x = spark_df.select(catcol).distinct().collect()
                for y in x:
                    categories.append(y[0])
                categories.sort()
                for k, category in enumerate(categories):
                    function = udf(lambda item: 1 if item == category else 0, IntegerType())
                    new_column_name = catcol + '_' + str(k)
                    spark_df = spark_df.withColumn(new_column_name, function(col(catcol)))
                spark_df = spark_df.drop(catcol)
            oldcols = list(spark_df.columns)
            spark_df = normalize(spark_df, list(spark_df.columns))
            normalcol = []
            for i in oldcols:
                normalcol.append(i + "_normalization")

            spark_df = spark_df.select(normalcol)
            column = normalcol
            clusalgolist = ["gm", "km",'bkm']
            kmlist = []
            bkmlist = []
            gmlist = []
            for c in clusalgolist:
                if c == "km":
                    for i in np.arange(2, 11):
                        km = KMeans().setK(i).setSeed(1)
                        model = km.fit(output)
                        clus = model.transform(result)
                        evaluator = ClusteringEvaluator()
                        silhouette = evaluator.evaluate(clus)
                        wssse = model.computeCost(output)
                        kmlist.append({"X": i, "Y1": silhouette, "Y2": wssse})
                elif c == "bkm":
                    for i in np.arange(2, 11):
                        km = BisectingKMeans().setK(i).setSeed(1)
                        model = km.fit(output)
                        clus = model.transform(result)
                        evaluator = ClusteringEvaluator()
                        silhouette = evaluator.evaluate(clus)
                        wssse = model.computeCost(output)
                        bkmlist.append({"X": i, "Y1": silhouette, "Y2": wssse})
                elif c == "gm":
                    for i in np.arange(2, 11):
                        km = GaussianMixture().setK(i).setSeed(1)
                        model = km.fit(output)
                        clus = model.transform(result)
                        evaluator = ClusteringEvaluator()
                        silhouette = evaluator.evaluate(clus)
                        gmlist.append({"X": i, "Y1": silhouette})
            t2 = datetime.now()
            connections.close_all()
            return Response({"status":"success","KMeans":kmlist,"BKMeans":bkmlist,"GMixture":gmlist})
        except Exception as e:
            traceback.print_exc()
            return Response({"status": "failure", "error": str(e)})

